In [1]:
import pandas as pd
import torch
import random
import numpy as np
from matplotlib import pyplot as plt
import torch.nn.functional as F
import torch.nn as nn

Collecting the data as categories on a csv file
For this notebook we are gonna take the Mnist dataset

In [2]:
data = np.array(pd.read_csv('datasets/train.csv'))
data = torch.tensor(data)


This part decides the category that is the output and the rest are taken as input 

In [3]:
y = data[:,0]
x = data[:,1:]/255   # pixel values reduced to range 0-1

Train Test Validation split

In [4]:
xtrain = x[:int(0.8*len(x))]
ytrain = y[:int(0.8*len(y))]
xval = x[int(0.8*len(x)):int(0.9*len(x))]
yval = y[int(0.8*len(y)):int(0.9*len(y))]
xest = x[int(0.9*len(x)):]
yest = y[int(0.9*len(y)):]

All the categories listed below with the number of training examles

In [5]:
xtrain.shape,ytrain.shape

(torch.Size([33600, 784]), torch.Size([33600]))

In [6]:
bat_sz = 4   # batch size
n_fl = 128   # number of neurons in the first layer
n_ml = 64    # number of neurons in the middle layer
n_ol = 10    # number of neurons in the output layer (Number of possible outcomes depicted as a neuron each)
ex_len = 784 # context length of one example
epoch = 10   # number of iterations on the entire dataset
l_r = 0.001  # learning rate
nl_r =0.0001 #rectified finetune stage learning rate

This model is the representation of fully connected deep neural network with the output neuron activations as the probability of the possible outputs

In [7]:
model = nn.Sequential(
    nn.Linear(ex_len,n_fl),
    #nn.ReLU(),
    nn.Tanh(),
    #nn.Sigmoid(),
    nn.Linear(n_fl,n_ml), 
    #nn.ReLU(),
    #nn.Tanh(),
    #nn.Sigmoid(),
    #nn.Dropout(0.0),
    nn.Linear(n_ml,n_ol),   # this layer will output the activations of 10 neurons (probability of the outputs)
    #nn.ReLU(),
    #nn.Tanh(),
    #nn.Sigmoid(),
    #nn.Softmax(),
)

ix = (torch.randperm(len(xtrain)-bat_sz))

Setting up the optimiser for the parameters

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=l_r)

In [9]:
def accuracy():    # Accuracy calculation function (on validation data)
    k,d=len(xval),len(xval)
    for n in range(len(xval)):
        omt = ((model(xval[n]))).tolist()
        k -= (omt.index(max(omt))==yval[n].item())
    accu = (((d-k)/d) *100)
    return("Accuracy = " + str(accu))

This iterator iterates throught all the randomised training examples and calculates the accuracy every epoch

In [10]:
for e in range(epoch):
    for i in ix:
        xt = xtrain[i:i+bat_sz]    # selecting the batch 
        yt = ytrain[i:i+bat_sz]     
        X_train = xt.view([bat_sz,ex_len])   # creating the appropriate tensor size
        Y_train = F.one_hot(yt,n_ol).float()   # creating a one hot vector representation of the possible output

        t = model(X_train.float())
        loss = F.cross_entropy(t,Y_train)    # Loss definition
        optimizer.zero_grad()    

        loss.backward()     # accumulating the gradient
        optimizer.step()    # rectifying the weights using the gradients
        
        # print(loss.item())
        
    print(str(e+1) + " / " + str(epoch)+", " + accuracy())
    if(e == int(0.8*epoch)):
        optimizer = torch.optim.SGD(model.parameters(), lr=nl_r)  # decreasing the learning rate 


1 / 10, Accuracy = 90.71428571428571
2 / 10, Accuracy = 92.38095238095238
3 / 10, Accuracy = 93.33333333333333
4 / 10, Accuracy = 94.0952380952381
5 / 10, Accuracy = 94.76190476190476
6 / 10, Accuracy = 95.21428571428572
7 / 10, Accuracy = 95.83333333333334
8 / 10, Accuracy = 96.16666666666667
9 / 10, Accuracy = 96.42857142857143
10 / 10, Accuracy = 96.33333333333334


This model performs really good on categorization datasets where you have a certain classes and you need to predict a class.
You can go ahead and run almost anything on it and get a decent accuracy.